<center>

# ***MVP de Machine Learning e Analytics - Uma abordagem multitarefa para a Criptoanálise Clássica: Uma homenagem aos Pioneiros das Redes Neurais Artificiais, de McCullogh e Pitts à Alan Turing***

> ## ***Decifrando a Bíblia Sagrada cifrada por César e Vigenère***



</center>

### ***Nome:*** **Luís Gabriel Nascimento Simas**

### ***Matrícula:*** **4052025000943**

# 0. **Apresentação**

## 0.1. **Temos atração pelo desconhecido**

## 0.2. **Por quê usar cifras?**

## 0.3. **Por quê a Bíblia?**

# 1. Definição do Problema

## 1.1. **Descrição do Problema**

# 2. **Aquisição e preparação dos dados**

## 2.1 **Aquisição**

### Baixando o repositório do github

In [128]:
!git clone https://github.com/gabrielsimas/biblia-cifra-cesar-vigenere.git

fatal: destination path 'biblia-cifra-cesar-vigenere' already exists and is not an empty directory.


### **Instalando os pacotes para o projeto**

In [129]:
import json
from dataclasses import dataclass, field
from typing import List, Optional
import random

### **Funções e Classes auxiliares**
### **Aqui estão todas as funções e classes utilizadas no Projeto**

In [130]:
BIBLIA_JSON_PATH = '/content/biblia-cifra-cesar-vigenere/KJA.json'

In [131]:
@dataclass
class Livro:
  abbrev: str
  chapters: List[List[str]]
  name: str

In [132]:
from os import lchown
@dataclass
class Biblia:
  livros: List[Livro] = field(default_factory=list)

  def carregar_arquivo_biblia_json(self, arquivo_json: str):
    with open(arquivo_json, 'r', encoding='utf-8') as f:
      dados_json = json.load(f)
      self.livros = [Livro(**livro) for livro in dados_json]

  def escolher_versiculo(self, livro_abbrev: Optional[str] = None, capitulo_num: Optional[int] = None) -> str:
    """
    Seleciona e retorna um versículo aleatório. Pode ser filtrado por livro e capítulo.

    Args:
        livro_abbrev (Optional[str]): A abreviação do livro (ex: 'Gn'). Se None, será aleatório.
        capitulo_num (Optional[int]): O número do capítulo (ex: 1). Se None, será aleatório.

    Returns:
        str: Um versículo formatado como "Livro Capítulo:Versículo Texto".
    """
    livros_filtrados = self.livros

    # 1. Escolhe o livro
    if livro_abbrev:
        livros_encontrados = [l for l in self.livros if l.abbrev.lower() == livro_abbrev.lower()]
        if not livros_encontrados:
            raise ValueError(f"Livro com abreviação '{livro_abbrev}' não encontrado.")
        livro_escolhido = livros_encontrados[0]
    else:
        livro_escolhido = random.choice(self.livros)

    # 2. Escolhe o capítulo
    capitulos_do_livro = livro_escolhido.chapters
    if capitulo_num:
        if 0 < capitulo_num <= len(capitulos_do_livro):
            capitulo_escolhido = capitulos_do_livro[capitulo_num - 1]
        else:
            raise ValueError(f"Capítulo {capitulo_num} não encontrado no livro de {livro_escolhido.abbrev}.")
    else:
        capitulo_escolhido = random.choice(capitulos_do_livro)
        capitulo_num = livro_escolhido.chapters.index(capitulo_escolhido) + 1

    # 3. Escolhe o versículo
    versiculo_escolhido = random.choice(capitulo_escolhido)
    numero_do_versiculo = capitulo_escolhido.index(versiculo_escolhido) + 1

    # Formata o output como "nome do livro numero do capitulo: número do versículo: texto do versículo"
    return f"{livro_escolhido.name} {capitulo_num}:{numero_do_versiculo}: {versiculo_escolhido}"


In [133]:
class Cifrador():
  def __init__(self, texto: str) -> None:
    self._texto_original = texto
    self._texto_atual = texto
    self._esta_cifrada = False

  @property
  def texto_atual(self) -> str:
    """Getter que retorna o texto no seu estado atual."""
    return self._texto_atual

  @texto_atual.setter
  def texto_atual(self, novo_texto: str):
    """Setter que atualiza o texto atual."""
    self._texto_atual = novo_texto

  @property
  def texto_original(self) -> str:
    """Getter que retorna o texto original."""
    return self._texto_original

  def converte_minuscula(self):
    """
    Converte o texto atual para minúsculas, mas apenas se ele não estiver cifrado.
    """
    if not self._esta_cifrada:
      self.texto_atual = self.texto_atual.lower()
    else:
      print("Erro: Não é possível converter para minúsculas. O texto já está cifrado.")

  def encode_cesar(self, shift: int) -> str:
    """
    Codifica o texto atual usando a Cifra de César, preservando a capitalização.
    """
    resultado = ""
    for char in self._texto_atual:
      if 'a' <= char <= 'z':
        nova_posicao = (ord(char) - ord('a') + shift) % 26
        resultado += chr(ord('a') + nova_posicao)
      elif 'A' <= char <= 'Z':
        nova_posicao = (ord(char) - ord('A') + shift) % 26
        resultado += chr(ord('A') + nova_posicao)
      else:
        resultado += char

    self.texto_atual = resultado
    self._esta_cifrada = True
    return self.texto_atual

  def encode_vigenere(self, chave: str) -> str:
    """
    Codifica o texto atual usando a Cifra de Vigenère, preservando a capitalização.
    """
    resultado = ""
    chave = chave.lower()
    indice_chave = 0

    for char in self._texto_atual:
      if 'a' <= char <= 'z':
        shift_vigenere = ord(chave[indice_chave]) - ord('a')
        nova_posicao = (ord(char) - ord('a') + shift_vigenere) % 26
        resultado += chr(ord('a') + nova_posicao)
        indice_chave = (indice_chave + 1) % len(chave)
      elif 'A' <= char <= 'Z':
        shift_vigenere = ord(chave[indice_chave]) - ord('a')
        nova_posicao = (ord(char) - ord('A') + shift_vigenere) % 26
        resultado += chr(ord('A') + nova_posicao)
        indice_chave = (indice_chave + 1) % len(chave)
      else:
        resultado += char

    self.texto_atual = resultado
    self._esta_cifrada = True
    return self._texto_atual

  def decode_cesar(self, shift: int) -> str:
    """
    Decodifica o texto atual usando a Cifra de César.
    """
    self._esta_cifrada = False
    return self.encode_cesar(-shift)

  def decode_vigenere(self, chave: str) -> str:
    """
    Decodifica o texto atual usando a Cifra de Vigenère.
    """
    resultado = ""
    chave = chave.lower()
    indice_chave = 0

    for char in self._texto_atual:
      if 'a' <= char <= 'z':
        shift_vigenere = ord(chave[indice_chave]) - ord('a')
        nova_posicao = (ord(char) - ord('a') - shift_vigenere) % 26
        resultado += chr(ord('a') + nova_posicao)
        indice_chave = (indice_chave + 1) % len(chave)
      elif 'A' <= char <= 'Z':
        shift_vigenere = ord(chave[indice_chave]) - ord('a')
        nova_posicao = (ord(char) - ord('A') - shift_vigenere) % 26
        resultado += chr(ord('A') + nova_posicao)
        indice_chave = (indice_chave + 1) % len(chave)
      else:
        resultado += char

    self._esta_cifrada = False
    self.texto_atual = resultado
    return self.texto_atual

  def reset(self) -> str:
    """
    Reseta o texto atual para o texto original.
    """
    self._esta_cifrada = False
    self.texto_atual = self.texto_original
    return self.texto_atual

In [134]:
import unicodedata
import re

def extrair_chave_da_citacao(texto_completo: str) -> str:
  """
    Extrai a citação de um versículo, remove a acentuação e outros caracteres, e retorna apenas as letras.

    Args:
        texto_completo (str): O versículo completo, incluindo a citação (ex: "3 João 1:3: ...").

    Returns:
        str: Apenas as letras da citação, em minúsculas e sem acentuação (ex: "joao").
  """
  match = re.search(r'^(.*?):', texto_completo)

  if match:
    citacao_bruta = match.group(1)
    citacao_sem_acento = unicodedata.normalize('NFKD', citacao_bruta).encode('ascii','ignore').decode('utf-8')
    chave_limpa = re.sub(r'[^a-zA-Z]','', citacao_sem_acento)
    return chave_limpa.lower()

  return ""

In [135]:
@dataclass
class AmostraDataset:
  """
  Representa uma única amostra de dados para o dataset.
  """
  texto_original: str
  texto_cifrado: str
  tipo_cifra: str
  chave_usada: Optional[str] = None

In [136]:
import pandas as pd

class GeradorConjuntoDados:
  def __init__(self, biblia: Biblia, cifrador: Cifrador) -> None:
    self._biblia = biblia
    self._cifrador = cifrador

  def _extrair_e_limpar_citacao(self, versiculo_completo: str) -> tuple[str, str]:
    """
    Extrai a citação de um versículo, remove a acentuação e outros caracteres, e retorna apenas as letras.

    Args:
        texto_completo (str): O versículo completo, incluindo a citação (ex: "3 João 1:3: ...").

    Returns:
        str: Apenas as letras da citação, em minúsculas e sem acentuação (ex: "joao").
    """
    partes = versiculo_completo.split(' ', 2)
    if len(partes) < 3:
      return "", ""

    citacao_completa = " ".join(partes[:2])
    texto_original = partes[2]

    citacao_sem_acento = unicodedata.normalize('NFKD', citacao_completa).encode('ascii','ignore').decode('utf-8')
    citacao_limpa = re.sub(r'[^a-zA-Z]', '', citacao_sem_acento)

    chave_final = citacao_limpa.lower()

    return chave_final, texto_original

  def _processar_cesar(self, texto_original: str) -> dict:
    self._cifrador.texto_atual = texto_original
    shift = random.randint(1, 25)
    texto_cifrado = self._cifrador.encode_cesar(shift)
    return AmostraDataset(
        texto_original=texto_original,
        texto_cifrado=texto_cifrado,
        tipo_cifra="cesar",
        chave_usada=str(shift)
    )

  def _processar_vigenere(self, texto_original: str, chave_base: str) -> dict:
    self._cifrador.texto_atual = texto_original
    shift_cesar_chave = random.randint(1, 25)

    chave_final = self._cifrador.encode_cesar(shift=shift_cesar_chave)
    texto_cifrado = self._cifrador.encode_vigenere(chave=chave_final)

    return AmostraDataset(
        texto_original=texto_original,
        texto_cifrado=texto_cifrado,
        tipo_cifra="cesar",
        chave_usada=chave_final
    )

  def gerar_conjunto_dados(self, nome_arquivo: str = "dataset_biblia_criptografada.parquet"):
    """
      Gera um dataset completo com todos os versículos da Bíblia,
      codificados com as cifras de César ou Vigenère, de forma aleatória,
      e salva o resultado em um arquivo Parquet.

      Args:
          nome_arquivo (str): O nome do arquivo Parquet a ser salvo.
    """
    amostras = []

    todos_os_versiculos = (versiculo for livro in self._biblia.livros
                            for capitulo in livro.chapters for versiculo in capitulo)

    for versiculo_completo in todos_os_versiculos:
      citacao_limpa, texto_original = self._extrair_e_limpar_citacao(versiculo_completo)

      if not text_original:
        continue

      cifra_escolhida = random.choice(["cesar", "vigenere"])

      if cifra_escolhida == "cesar":
        amostra = self._processar_cesar(texto_original)
      else:
        amostra = self._processar_vigenere(texto_original, citacao_limpa)

      amostras.append(amostra)

    df_dataset = pd.DataFrame(amostras)
    df_dataset.to_parquet(nome_arquivo, index=False)
    print(f"Dataset gerado e salvo com sucesso em '{nome_arquivo}'!")


## 2.2. **Preparação dos dados**

#### **Criação do dataset**

In [137]:
biblia_obj = Biblia()
cifrador_obj = Cifrador("")

In [138]:
biblia_obj.carregar_arquivo_biblia_json(BIBLIA_JSON_PATH)

In [139]:
gerador = GeradorConjuntoDados(biblia=biblia_obj, cifrador=cifrador_obj)
gerador.gerar_conjunto_dados(nome_arquivo='/content/biblia-cifra-cesar-vigenere/dataset_biblia_criptografada.parquet')

Dataset gerado e salvo com sucesso em '/content/biblia-cifra-cesar-vigenere/dataset_biblia_criptografada.parquet'!


# 3. **Pré-Processamento**

# 4. **Modelagem e Inferência**

# **Conclusão**